## Univariate one step problem with mlp

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# Define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=5)

# Print each data sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

# Define model
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Fit model
model.fit(generator, steps_per_epoch=1, epochs=200, verbose=0)

# Make a one step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input))
print(x_input)
y_output = model.predict(x_input, verbose=0)
print(y_output)

[[1 2]
 [2 3]
 [3 4]
 [4 5]
 [5 6]] => [3 4 5 6 7]
[[6 7]
 [7 8]
 [8 9]] => [ 8  9 10]
[[ 9 10]]
[[11.451763]]


## Univariate one step problem with LSTM

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# Define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series)

# Reshape to [10, 1]
n_features = 1
series = series.reshape((len(series), n_features))
print(series)

# Define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=5)

# Print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

# Define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Fit model
model.fit(generator, steps_per_epoch=1, epochs=400, verbose=0)

# Make a one step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input, n_features))
y_output = model.predict(x_input, verbose=0)
print(y_output)

[ 1  2  3  4  5  6  7  8  9 10]
[[ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]]
[[[1]
  [2]]

 [[2]
  [3]]

 [[3]
  [4]]

 [[4]
  [5]]

 [[5]
  [6]]] => [[3]
 [4]
 [5]
 [6]
 [7]]
[[[6]
  [7]]

 [[7]
  [8]]

 [[8]
  [9]]] => [[ 8]
 [ 9]
 [10]]
[[11.8065]]


## Multivariate one step problem with LSTM - same data for input and output



In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Define generator
n_features = dataset.shape[1]
print("n_features = %d" % n_features)
n_input = 2
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=5)

# Print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

# Define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
# Fit model
model.fit(generator, steps_per_epoch=1, epochs=400, verbose=0)

# Make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
y_output = model.predict(x_input, verbose=0)
print(y_output)

[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
n_features = 2
[[[10 15]
  [20 25]]

 [[20 25]
  [30 35]]

 [[30 35]
  [40 45]]

 [[40 45]
  [50 55]]

 [[50 55]
  [60 65]]] => [[30 35]
 [40 45]
 [50 55]
 [60 65]
 [70 75]]
[[[60 65]
  [70 75]]

 [[70 75]
  [80 85]]

 [[80 85]
  [90 95]]] => [[ 80  85]
 [ 90  95]
 [100 105]]
[[111.20885 116.2134 ]]



## Multivariate one step problem - different data for input and output


In [ ]:
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])
# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Define generator
n_input = 1
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
[[[10 15]]] => [[45]]
[[[20 25]]] => [[65]]
[[[30 35]]] => [[85]]
[[[40 45]]] => [[105]]
[[[50 55]]] => [[125]]
[[[60 65]]] => [[145]]
[[[70 75]]] => [[165]]
[[[80 85]]] => [[185]]
[[[90 95]]] => [[205]]


In [ ]:
# multivariate one step problem
from numpy import array
from numpy import hstack
from numpy import insert
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# Define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])
# Reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# Shift the target sample by one step
out_seq = insert(out_seq, 0, 0)
print(out_seq)
out_seq = out_seq[:-1]
print(out_seq)

# Define generator
n_input = 1
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
[  0  25  45  65  85 105 125 145 165 185 205]
[  0  25  45  65  85 105 125 145 165 185]
[[[10 15]]] => [25]
[[[20 25]]] => [45]
[[[30 35]]] => [65]
[[[40 45]]] => [85]
[[[50 55]]] => [105]
[[[60 65]]] => [125]
[[[70 75]]] => [145]
[[[80 85]]] => [165]
[[[90 95]]] => [185]


 ## Univariate multi-step problem

In [ ]:
from numpy import array
from keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
target = array([[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9,10],[10,11]])
# define generator
n_input = 2
generator = TimeseriesGenerator(series, target, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

[[1 2]] => [[3 4]]
[[2 3]] => [[4 5]]
[[3 4]] => [[5 6]]
[[4 5]] => [[6 7]]
[[5 6]] => [[7 8]]
[[6 7]] => [[8 9]]
[[7 8]] => [[ 9 10]]
[[8 9]] => [[10 11]]
